In [62]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init


In [63]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [64]:
#parmeters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [65]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download=True)
nmist_test = dsets.MNIST(root='MNIST_data/',
                          train = False,
                          transform = transforms.ToTensor(),
                          download=True)

In [66]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last=True)

In [67]:
class CNN(nn.Module):

  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(32,64,kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.fc = nn.Linear(7*7*64,10,bias=True)
    torch.nn.init.xavier_uniform_(self.fc.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)

    out = out.view(out.size(0), -1)
    out = self.fc(out)
    return out


In [68]:
model = CNN().to(device)

In [69]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [70]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)

    cost = criterion(hypothesis, Y)
    cost.backward()

    optimizer.step()

    avg_cost += cost/total_batch

  print('[Epoch: {}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished')

[Epoch: 1] cost = 0.2255837619304657
[Epoch: 2] cost = 0.06296368688344955
[Epoch: 3] cost = 0.04615773260593414
[Epoch: 4] cost = 0.03741168975830078
[Epoch: 5] cost = 0.031386978924274445
[Epoch: 6] cost = 0.02602524124085903
[Epoch: 7] cost = 0.021933691576123238
[Epoch: 8] cost = 0.018370728939771652
[Epoch: 9] cost = 0.016225866973400116
[Epoch: 10] cost = 0.013150853104889393
[Epoch: 11] cost = 0.009825163520872593
[Epoch: 12] cost = 0.00980664324015379
[Epoch: 13] cost = 0.007727202959358692
[Epoch: 14] cost = 0.006907863076776266
[Epoch: 15] cost = 0.006836413871496916
Learning Finished
